In [ ]:
!pip install torchtext==0.8

     |████████████████████████████████| 6.9 MB 1.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0


Load Package

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


from torchtext.datasets import TranslationDataset

from spacy.lang.en import English
from spacy.lang.de import German
from spacy.lang.fr import French
import spacy
import numpy as np

import random
import math
import time
from torchtext.data import Field,TabularDataset,Iterator,BucketIterator

import unicodedata
import string
import re
import random

In [ ]:
## Mount Google Drive Data (If using Google Colaboratory)
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
except:
    print("Mounting Failed.")
    

Mounted at /content/gdrive


Process zip file

In [ ]:
import zipfile
import os
from tqdm import tqdm, trange

! tar xvf /content/gdrive/MyDrive/Colab\ Notebooks/ml/hansard.36.r2001-1a.house.debates.training.tar
! tar xvf /content/gdrive/MyDrive/Colab\ Notebooks/ml/hansard.36.r2001-1a.house.debates.testing.tar
% cd /content/hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/testing/1
! gunzip *.gz
% cd /content/hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/testing/2
! gunzip *.gz
% cd /content/hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training
! gunzip *.gz


hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.001.f.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.002.e.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.002.f.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.003.e.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.003.f.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.004.e.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.004.f.gz
hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/hansard.36.1.house.debates.005.e.gz
hansard.36/Release

Load data

In [ ]:
dir = '/content/hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/{}ing/'
def load_data(mode):
    if mode == 'train':
        file_dir = dir.format('train')
    elif mode == 'dev':
        file_dir = dir.format('test') + '1/'
    elif mode == 'test':
        file_dir = dir.format('test') + '2/'
    else:
        raise Exception("select mode: train / dev / test")
        
    filenames = []
    fr_files = []
    en_files = []
    # for filename in os.listdir(file_dir):
    for filename in os.listdir(file_dir):
        if filename.split('.')[-1] == 'e':
          myfile = '.'.join(filename.split('.')[:-1])
          fr_files.append(myfile+'.f')
          en_files.append(myfile+'.e')
        # if filename.split('.')[-1] == 'e':
        #   en_files.append(filename)
        # elif filename.split('.')[-1] == 'f':
        #   fr_files.append(filename)
    #     filenames.append(filename)
    # filenames = sorted(filenames)
    # fr_files = filenames[0::2]
    # en_files = filenames[1::2]
    
    en_data = load_from_sorted_files(en_files, file_dir)
    fr_data = load_from_sorted_files(fr_files, file_dir)
    print('en sentences:', len(en_data))
    print('fr sentences:', len(fr_data))
    return [en_data, fr_data]

def load_from_sorted_files(files, file_dir):
    dataset = []
    for file in tqdm(files):
        try:
          with open(file_dir+file, 'r', encoding='latin-1') as f:
              lines = f.readlines()
              for i in range(len(lines)):
                  line = lines[i]
                  line = line.strip()

                  dataset.append(line)
        except:
          continue
    return dataset
en_data_tr, fr_data_tr = load_data('train')
en_data_dev, fr_data_dev = load_data('dev')
en_data_te, fr_data_te = load_data('test')

100%|██████████| 313/313 [00:00<00:00, 592.28it/s]


en sentences: 947969
fr sentences: 947969


100%|██████████| 22/22 [00:00<00:00, 683.33it/s]


en sentences: 62213
fr sentences: 62213


100%|██████████| 15/15 [00:00<00:00, 524.90it/s]

en sentences: 60077
fr sentences: 60077


Tokenizer

In [ ]:
spacy_en = English()
spacy_fr = French()
def tokenize_fr(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

Define field

In [ ]:
SRC_field = Field(tokenize = tokenize_fr, init_token = '<sos>', eos_token = '<eos>', lower = True, include_lengths = True)
TRG_field = Field(tokenize = tokenize_en, init_token = '<sos>',  eos_token = '<eos>',  lower = True)

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


preprocess data.

In [ ]:
import pandas as pd

d= []
count = 0
for i in range(len(en_data_tr)):
  # if count>100:
  #   break

  if len(en_data_tr[i].split(' '))>=60:
    continue
  # count += 1
  d.append([fr_data_tr[i],en_data_tr[i]])

train = pd.DataFrame(data=d,
             columns = ['src','trg'])
train.to_csv("./train.csv", index=False)

d= []
for i in range(len(en_data_dev)):
  # if i>100:
  #   break
  if len(en_data_dev[i].split(' '))>=60:
    continue
  d.append([fr_data_dev[i],en_data_dev[i]])

test = pd.DataFrame(data=d,
             columns = ['src','trg'])
test.to_csv("./val.csv", index=False)

d= []
for i in range(len(en_data_te)):
  # if i>100:
  #   break
  if len(en_data_te[i].split(' '))>=60:
    continue
  d.append([fr_data_te[i],en_data_te[i]])

test = pd.DataFrame(data=d,
             columns = ['src','trg'])
test.to_csv("./test.csv", index=False)
fields = [('src', SRC_field), ('trg', TRG_field)]

train_data, valid_data, test_data= TabularDataset.splits(path='./', train='train.csv', validation='val.csv', test = 'test.csv', format='csv', fields=fields)

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


build the vocabulary.

In [ ]:
SRC_field.build_vocab(train_data, min_freq = 2)
TRG_field.build_vocab(train_data, min_freq = 2)

Build dataloader

In [ ]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.src),
     device = device)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)




Define Encoder, Attention, Decoder and Seq2Seq

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len): 
        embedded = self.dropout(self.embedding(src))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.to('cpu'))  
        packed_outputs, hidden = self.rnn(packed_embedded)      
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        return outputs, hidden
  
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attention_function = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.linear_function = nn.Linear(dec_hid_dim, 1, bias = False)
        self.softmax = torch.nn.Softmax(dim = 1)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]

        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        energy_ = torch.tanh(self.attention_function(torch.cat((hidden, encoder_outputs), dim = 2))) 
        attention = self.linear_function(energy_).squeeze(2)
        attention = attention.masked_fill(mask == 0, -1e15)
        return self.softmax(attention)

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        a = self.attention(hidden, encoder_outputs, mask)

        a = a.unsqueeze(1)

        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        
        weighted = torch.bmm(a, encoder_outputs)
        
        weighted = weighted.permute(1, 0, 2)

        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src, src_len)
        input = trg[0,:]
        
        mask = self.create_mask(src)

                
        for t in range(1, trg_len):

            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)
            
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            if random.random() < teacher_forcing_ratio:
              input = trg[t]
            else:
              input = output.argmax(1)      
        return outputs

Initialization

In [ ]:
INPUT_DIM = len(SRC_field.vocab)
OUTPUT_DIM = len(TRG_field.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
SRC_PAD_IDX = SRC_field.vocab.stoi[SRC_field.pad_token]

attention_model = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attention_model)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)
TRG_PAD_IDX = TRG_field.vocab.stoi[TRG_field.pad_token]
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

Train and evaluate

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    loss_epoch = 0
    
    for i, batch in enumerate(iterator):
        
        src_data, src_len = batch.src
        trg_data = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src_data, src_len, trg_data)
        output = output[1:].view(-1, output.shape[-1])
        
        trg_data = trg_data[1:].view(-1)
        
        loss = criterion(output, trg_data)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        loss_epoch += loss.item()
        
    return loss_epoch / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    loss_epoch = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src_data, src_len = batch.src
            trg_data = batch.trg

            output = model(src_data, src_len, trg_data, 0) 
            output = output[1:].view(-1, output.shape[-1])
            trg_data = trg_data[1:].view(-1)

            loss = criterion(output, trg_data)
            loss_epoch += loss.item()
        
    return loss_epoch / len(iterator)

In [ ]:
def cal_time_cost(start, end):
    cost = end - start
    minutes = int(cost / 60)
    seconds = int(cost - (minutes * 60))
    return minutes, seconds

Train our model

In [ ]:
N_EPOCHS = 8
CLIP = 1

best_valid_loss = float('inf')
total_train_loss = []
total_valid_loss = []

for epoch in range(N_EPOCHS):
    
    start = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end = time.time()
    
    epoch_mins, epoch_secs = cal_time_cost(start, end)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/gdrive/MyDrive/Colab Notebooks/ml/RNN__Based_Model.pt')
    
    print(f'Epoch: {epoch+1:02}, Time: {epoch_mins}m {epoch_secs}s')
    print(f',Train Loss is: {train_loss:.3f}')
    print(f',Validation Loss: {valid_loss:.3f}')

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01, Time: 101m 54s
,Train Loss is: 2.815
,Validation Loss: 3.349
Epoch: 02, Time: 101m 51s
,Train Loss is: 2.203
,Validation Loss: 3.347
Epoch: 03, Time: 101m 51s
,Train Loss is: 2.053
,Validation Loss: 3.314
Epoch: 04, Time: 101m 50s
,Train Loss is: 1.967
,Validation Loss: 3.282
Epoch: 05, Time: 101m 48s
,Train Loss is: 1.909
,Validation Loss: 3.337
Epoch: 06, Time: 101m 58s
,Train Loss is: 1.862
,Validation Loss: 3.402
Epoch: 07, Time: 101m 47s
,Train Loss is: 1.826
,Validation Loss: 3.332
Epoch: 08, Time: 102m 9s
,Train Loss is: 1.799
,Validation Loss: 3.362


Test our model on test set

In [ ]:
model.load_state_dict(torch.load('/content/gdrive/MyDrive/Colab Notebooks/ml/RNN__Based_Model.pt'))
test_loss = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} ')

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Test Loss: 2.378 


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)])
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_len)

    mask = model.create_mask(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)

        attentions[i] = attention
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attentions[:len(trg_tokens)-1]

Calculate BLEU Score

In [ ]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

In [ ]:
bleu_score = calculate_bleu(test_data, SRC_field, TRG_field, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

BLEU score = 37.10


We get a BLEU of 37.1